In [88]:
#re.findall('\(.*?\)',b1)
re.findall('\(.*?\)',f1)

['(2004)']

In [296]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

In [308]:
wikipages = ['https://en.wikipedia.org/wiki/List_of_children%27s_books_made_into_feature_films?oldformat=true',
             'https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(0%E2%80%939,_A%E2%80%93C)?oldformat=true',
             'https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(D%E2%80%93J)?oldformat=true',
             'https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(K%E2%80%93R)?oldformat=true',
             'https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(S%E2%80%93Z)?oldformat=true'
            ]

In [303]:
def add_to_rows(page_soup):
    '''
    Method extracts row elements from the soup tags of individual pages 
    and appends them to the `rows` list.
    
    '''
    
    global rows
    
    tables = page_soup.find_all('table', {'class': 'wikitable'})

    for alphabet in tables:
        rows += alphabet.findAll('tr')
    
    print("Tables added: {}, Rows added: {}".format(len(tables), len(rows)))

In [306]:
def add_to_book_film_dict():
    '''
    Method parses newly updated rows for book/film title, book author,
    number of adaptions, and list of adaptations, and adds them to the
    `book_film_dict` as a nested dictionary. 
    '''
    global sum_adaptations
    
    for row in rows:
        cols = row.findAll('td')

        try:
            book_title_year = cols[0].text.replace('\n', '').split(',')[0]
            author = cols[0].text.replace('\n', '').split(',')[-1].lstrip() 
    
            adaptations = cols[1].text.replace('\n', '').rstrip()
            adaptations = list(adaptations.split(')'))
    
            for index, title in enumerate(adaptations):
                if ('TV' in title):
                    adaptations[index - 1] += ' (TV)'
                    del adaptations[index]
            
                if len(title) > 0:
                    adaptations[index] += ')'
        
            adaptations = adaptations[:len(adaptations) - 1]
            
            sum_adaptations += len(adaptations)
                 
            book_film_dict[book_title_year] = {'author': author, 
                                               'count': len(adaptations), 
                                               'adaptations': adaptations}
        except IndexError:
            pass
        
    print("No. of rows/original books in dict: {}, No. of adaptations: {} \n"
          .format(len(book_film_dict), sum_adaptations))

In [312]:
rows = []
book_film_dict = {}
sum_adaptations = 0

for url in wikipages:
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'lxml')
    
    add_to_rows(soup)
    add_to_book_film_dict()
    
print("Of the {} rows parsed, {} were added to the dictionary ~ {}%"
      .format(len(rows), len(book_film_dict), (len(book_film_dict) * 100/len(rows))))

Tables added: 25, Rows added: 376
No. of rows/original books in dict: 348, No. of adaptations: 1223 

Tables added: 3, Rows added: 798
No. of rows/original books in dict: 732, No. of adaptations: 3220 

Tables added: 7, Rows added: 1245
No. of rows/original books in dict: 1143, No. of adaptations: 6088 

Tables added: 8, Rows added: 1662
No. of rows/original books in dict: 1513, No. of adaptations: 9894 

Tables added: 7, Rows added: 2034
No. of rows/original books in dict: 1845, No. of adaptations: 14605 

Of the 2034 rows parsed, 1845 were added to the dictionary ~ 90.70796460176992%
